In [1]:
# Importing required packages

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import time
import datetime
from datetime import datetime
import socket
import getpass
import html2text as h2h
from html.parser import HTMLParser

#required packages
import mysql.connector as connection
from functools import reduce


In [2]:
st = []
ed = []
du = []
date_range= []
v_host_name = []

pd.set_option('display.max_column', None)

start = datetime.now()
st.append(start)

## Final Function
def f_final_function(p_start_date, p_end_date):
    
    try:
        v_visit_started_date = p_start_date
        v_visit_ended_date = p_end_date
        
        def f_connect_to_database(p_host, p_user, p_password, p_database):
            while True:
                try:
                    # Setup database connection
                    v_mydb = connection.connect(
                        host=p_host,
                        user=p_user,
                        password=p_password,
                        database=p_database,
                        use_pure=True
                    )
                    v_cursor = v_mydb.cursor()
                    print("Connected to the database\n")
                    return v_mydb, v_cursor

                except connection.Error as err:
                    print(f"Error: {err}")
                    print("\n retry !")
                    return None, None


        # Function to close the database connection
        def f_close_database_connection(p_mydb, p_cursor):
            try:
                if p_cursor is not None:
                    p_cursor.close()
                    print("Cursor closed.")
                if p_mydb.is_connected():
                    p_mydb.close()
                    print("Database connection closed.\n")
            except Exception as e:
                return f"An error occurred while closing the database connection: {e}"

        while True:
            v_host_input = input("Enter the host: ")
            v_user_input = input("Enter the username: ")
            v_password_input = getpass.getpass("Enter the password: ")
            v_database_input = input("Enter the database: ")
            v_mydb,v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

            if v_mydb is not None:

                #Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
                break

        # Query 1
        v_queries = {'Query1':'''SELECT v.visit_id AS Visit_id,
                    v.patient_id AS Patient_id,
                    MAX(pi.identifier) AS OpenMRS_id,
                    MAX(CONCAT(pn.given_name, ' ', pn.family_name)) as Patient_name,
                    p.gender AS Gender,
                    p.birthdate AS Date_of_birth,
                    YEAR(CURDATE()) - YEAR(p.birthdate) AS Age,
                    u.username as CHW_name,
                    v.date_Started AS Visit_started_date,
                    v.date_stopped AS Visit_ended_date,
                    v.date_created AS Visit_creation_date,
                    v.creator AS Creator_Id,
                    l.city_village as Location_village,
                    ls.name AS Location_sanch,
                    ld.name AS Location_district,
                    l.state_province as Location_state
                FROM visit v
                JOIN person p ON v.patient_id = p.person_id
                JOIN person_name pn ON v.patient_id=pn.person_id
                LEFT JOIN location l ON v.location_id=l.location_id
                JOIN users u ON u.user_id=v.creator
                JOIN patient_identifier pi ON pi.patient_id=v.patient_id
                LEFT JOIN location ls on (ls.location_id = l.parent_location)
                LEFT JOIN location ld on (ld.location_id = ls.parent_location )
                LEFT JOIN location lst on (lst.location_id = ld.parent_location )
                WHERE v.voided = 0
                AND p.voided=0
                AND pn.preferred = 1
                AND pn.voided=0
                AND pi.identifier_type=3
                AND  pi.voided=0
                AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id, v.date_Started, v.date_created,
                     v.date_stopped, v.creator, v.patient_id,
                     pi.identifier, l.state_province, l.city_village,
                    CASE WHEN l.parent_location IS NOT NULL THEN  l.name ELSE l.city_village END ,
                    CASE WHEN l.parent_location IS NOT NULL THEN  ls.name ELSE NULL END ,
                    CASE WHEN l.parent_location IS NOT NULL THEN  ld.name ELSE l.county_district END ,
                    CASE WHEN l.parent_location IS NOT NULL THEN  lst.name ELSE l.state_province END;''',

        'Query2':'''SELECT v.visit_id as Visit_id,
                    e.creator AS Doctor_id
                    FROM visit v
                    JOIN encounter e ON e.visit_id=v.visit_id 
                    WHERE v.voided=0
                          AND e.encounter_type=9
                          AND e.voided=0
                          AND v.date_Started BETWEEN %s AND %s;''',

        'Query3':'''SELECT v.visit_id as Visit_id,
                    MAX(CASE WHEN o.concept_id = 5085 THEN o.value_numeric END) AS Sbp,
                    MAX(CASE WHEN o.concept_id = 5086 THEN o.value_numeric END) AS Dbp,
                    MAX(CASE WHEN o.concept_id = 5087 THEN o.value_numeric END) AS Pulse,
                    MAX(CASE WHEN o.concept_id = 5088 THEN o.value_numeric END) AS Temperature,
                    MAX(CASE WHEN o.concept_id = 5089 THEN o.value_numeric END) AS Weight,
                    MAX(CASE WHEN o.concept_id = 5090 THEN o.value_numeric END) AS Height,
                     MAX(CASE WHEN o.concept_id = 5242 THEN o.value_numeric END) AS RR,
                    MAX(CASE WHEN o.concept_id = 5092 THEN o.value_numeric END) AS SPO2,
                    MAX(CASE WHEN o.concept_id = 165175 THEN o.value_text END) AS HB,
                    -- MAX(CASE WHEN o.concept_id = 163177 THEN o.value_numeric END) AS Uric_acid,
                    -- MAX(CASE WHEN o.concept_id = 1006 THEN o.value_numeric END) AS Total_cholestrol,
                    MAX(CASE WHEN o.concept_id = 165178 THEN o.value_text END) AS Sugar_random,
                    MAX(CASE WHEN o.concept_id = 165179 THEN o.value_text END) AS Blood_group,
                    MAX(CASE WHEN o.concept_id = 165177 THEN o.value_text END) AS Sugar_pp,
                    MAX(CASE WHEN o.concept_id = 165176 THEN o.value_text END) AS Sugar_after_meal,
                    MAX(CASE WHEN o.concept_id = 162169 THEN o.value_text END) AS Additional_comments
                FROM visit v
                JOIN encounter e ON e.visit_id=v.visit_id
                JOIN obs o ON o.person_id = v.patient_id
                WHERE v.voided=0 
                      AND e.encounter_type=6
                      AND e.voided=0
                      AND o.voided=0
                      AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id
                ORDER BY v.visit_id;''',

        'Query4':'''SELECT v.visit_id as Visit_id,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163212 THEN o.value_text ELSE NULL END) AS Chief_complaint,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163213 THEN o.value_text ELSE NULL END) AS Physical_examination,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163210 THEN o.value_text ELSE NULL END) AS Patient_medical_history,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163211 THEN o.value_text ELSE NULL END) AS Family_history
                FROM visit v
                JOIN encounter e ON v.visit_id = e.visit_id
                JOIN obs o ON e.encounter_id = o.encounter_id
                WHERE v.voided = 0 
                    AND e.encounter_type = 1
                    AND e.voided = 0
                    AND o.voided = 0
                    AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id
                ORDER BY v.visit_id;''',

        'Query5':'''SELECT v.visit_id as Visit_id,
                       CASE WHEN MAX(CASE WHEN o.concept_id = 163219 THEN o.value_text END) IS NOT NULL THEN 'yes' ELSE 'no' END AS Diagnosis_provided,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163219 THEN o.value_text ELSE NULL END) AS Diagnosis,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163202 THEN o.value_text ELSE NULL END SEPARATOR ';') AS Medications,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163206 THEN o.value_text ELSE NULL END) AS Medical_test,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163205 THEN o.value_text ELSE NULL END) AS Medical_advice,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 165095 THEN o.value_text ELSE NULL END) AS Notes,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163343 THEN o.value_text ELSE NULL END) AS Feedback,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163344 THEN o.value_text ELSE NULL END) AS Comments,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163345 THEN o.value_text ELSE NULL END) AS Follow_up_date
                FROM visit v
                JOIN encounter e ON e.visit_id=v.visit_id
                JOIN obs o ON o.encounter_id=e.encounter_id
                WHERE v.voided=0
                      AND e.encounter_type=9
                      AND e.voided=0
                      AND o.voided=0
                      AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id
                ORDER BY v.visit_id;''',

        'Query6':'''SELECT v.visit_id as Visit_id,
               MAX(CASE WHEN va.attribute_type_id = 5 THEN va.value_reference END) AS Specialty,
                MAX(CASE WHEN va.attribute_type_id = 7 THEN va.value_reference END) AS Visit_upload_time
                FROM visit v
                JOIN visit_attribute va ON va.visit_id=v.visit_id
                WHERE v.voided=0
                      AND (va.attribute_type_id=5 OR va.attribute_type_id=7)
                      AND va.voided=0
                      AND v.date_Started BETWEEN %s AND %s
               GROUP BY v.visit_id
            ORDER BY v.visit_id;''',


        'Query7':'''SELECT v.visit_id as Visit_id,
                       MAX(distinct CASE WHEN e.encounter_type = 9 THEN e.encounter_datetime ELSE NULL END) AS Consultation_start_time,
                       MAX(CASE WHEN e.encounter_type = 14 THEN CONCAT(pn.given_name, ' ', pn.family_name) END) AS Doctor_name,
                       TIME_TO_SEC(SEC_TO_TIME(HOUR(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime
                                  ELSE NULL END),v.date_created))*3600 +
                                  (MINUTE(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime
                                  ELSE NULL END),v.date_created))*60) +
                                  (SECOND(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime
                                  ELSE NULL END),v.date_created)))))/ 60 AS Doctor_TAT,
                       TIME_TO_SEC(SEC_TO_TIME(HOUR(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN v.date_stopped
                                  ELSE e.encounter_datetime END), v.date_started))*3600 +
                                  (MINUTE(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN v.date_stopped
                                  ELSE e.encounter_datetime END), v.date_started))*60)+
                                  (SECOND(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN v.date_stopped
                                  ELSE e.encounter_datetime END), v.date_started)))))/ 60 AS Patient_TAT,
              MAX(distinct CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime ELSE NULL END) AS Sign_submit_time
                FROM visit v
                JOIN encounter e ON e.visit_id=v.visit_id
                JOIN users u ON u.user_id=e.creator
                JOIN person_name pn ON pn.person_id=u.person_id
                WHERE v.voided=0
                      AND e.voided=0
                      AND pn.voided=0
                      AND v.date_Started BETWEEN %s AND %s
                GROUP by v.visit_id;'''}

        # Here append all the quries
        #v_query2 = [q1,q2,q3,q4,q5,q6,q7]


        def f_get_date_range_from_tables(p_mydb, p_cursor):
            try:
                if p_mydb.is_connected():
                    # SQL query to fetch the minimum and maximum dates from "visit" table
                    date_query = "SELECT MIN(date_Started) AS min_date, MAX(date_Started) AS max_date FROM visit WHERE voided = 0;"

                    p_cursor.execute(date_query)
                    v_date_results = p_cursor.fetchall()

                    if v_date_results:
                        v_min_date, v_max_date = v_date_results[0]

                        return v_min_date, v_max_date

                    else:
                        print("No data available in the tables.")
                        return None, None
                else:
                    print("Database connection is not opened.")
                    return None, None
            except Exception as e:
                print(f"An error occurred: {e}")


        # Connect to the database
        v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

        # Get the date range from the SQL tables
        v_min_table_date, v_max_table_date = f_get_date_range_from_tables(v_mydb, v_cursor)

        # Close the database connection
        f_close_database_connection(v_mydb, v_cursor)

        # Print the date range to inform the user
        #print(f"Date range available in the SQL table: {v_min_table_date} to {v_max_table_date}\n")

        # Function to execute a query with date input parameters
        def f_execute_query_with_params(p_mydb, p_cursor, p_query, p_params):
            try:
                if p_mydb.is_connected():
                    p_cursor.execute(p_query, p_params)
                    v_results = p_cursor.fetchall()
                    v_columns = [desc[0] for desc in p_cursor.description]
                    df = pd.DataFrame(v_results, columns=v_columns)

                    return df
                else:
                    print("Database connection is not opened.")
                    return None
            except Exception as e:
                print(f"An error occurred: {e}")
                return None

        #Creating a function that will extract the data before the associated symptoms
        def f_extract_section(p_text):
            try:
                v_matches = re.findall(r'(.*?)Associated symptoms', p_text, re.DOTALL)
                if v_matches:
                    return v_matches[0].strip()
                else:
                    return p_text.strip()
            except Exception as e:
                print(f"Error: {e}")
                return p_text.strip()


        # Extracting the Chief complaint from the data
        def f_extract_all_symptoms(p_text):
            try:
                v_matches = re.findall(r'►<b>(.*?)</b>', p_text)
                if v_matches:
                    v_formatted_symptoms = ', '.join(v_matches)
                    return v_formatted_symptoms
            except:
                print("An error occurred while extracting symptoms.")
            return None


        #Combining both the functions to extract the all the chief complaints
        def f_combined_extraction(p_text):
            v_section = f_extract_section(p_text)
            if v_section:
                return f_extract_all_symptoms(v_section)
            return None


        # Extracting the associated symptoms from the visit samples
        def f_extract_associated_symptoms(p_text):
            try:
                v_pattern = r'Associated symptoms</b>: <br/>• Patient reports -<br/>(.*?)<br/>'
                v_matches = re.findall(v_pattern, p_text, re.DOTALL)

                v_associated_symptoms = []
                if v_matches:
                    v_cleaned_text = re.sub(r'<.*?>', '', v_matches[0]).strip().replace('•', '')
                    v_cleaned_text = v_cleaned_text.replace('Patient denies', '').strip()
                    v_associated_symptoms = [part.split('-')[0].strip() for part in v_cleaned_text.split(',')]

                v_formatted_symptoms = ', '.join(v_associated_symptoms)
                return v_formatted_symptoms if v_formatted_symptoms else 'Null'
            except Exception as e:
                # Handle the exception here, e.g., log the error or return a default value
                return 'An error occurred: {}'.format(e)

        # Creating a function for parsing the data
        def f_extracting_strength(p_data, p_visit_id, col_index):
            v_data = p_data
            try:
                # Creating an empty list for saving the parsed data
                v_dosage = []
                # Parsing the data and extracting dosages
                for idx, medication in enumerate(v_data.split(';')):
                    try:
                        # Splitting medication by colon to separate name and dosage
                        parts = medication.split(':')
                        if len(parts) > 1:
                            # Extracting dosage from the second part after colon
                            dosage_part = parts[1].strip()
                            # Removing any commas and extra spaces
                            dosage = dosage_part.split(',')[0].strip()
                            v_dosage.append(dosage)
                        else:
                            # If no colon, consider dosage as null
                            v_dosage.append('null')
                    except IndexError:
                        print(f"Error processing strength for visit_id {p_visit_id}: {medication}. Skipping.")
                # Joining the extracted dosages into a single string
                return ', '.join(v_dosage)
            except Exception as e:
                print(e)

        # Creating a function for parsing the data
        def f_extracting_dosage(p_data, p_visit_id, col_index):
            v_data = p_data
            try:
                # Creating an empty list for saving the parsed data
                v_dosage = []
                # Parsing the data and extracting dosages
                for idx, medication in enumerate(v_data.split(';')):
                    try:
                        # Splitting medication by colon to separate name and dosage
                        v_parts = medication.split(':')
                        if len(v_parts) > 1:
                            # Extracting dosage from the second part after colon
                            #dosage_part = parts[1].strip()
                            # Removing any commas and extra spaces
                            #dosage = dosage_part.split(',')[1].strip()
                            dosage = ', '.join(v_parts[1].split(',')[1:]).strip()  # Extracting all elements from the second element onwards
                            v_dosage.append(dosage)
                        else:
                            # If no colon, consider dosage as null
                            v_dosage.append('null')
                    except IndexError:
                        print(f"Error processing dosage for visit_id {p_visit_id}: {medication}. Skipping.")
                # Joining the extracted dosages into a single string
                return '. '.join(v_dosage)
            except Exception as e:
                print(e)

        # Creating a function for parsing the data
        def f_extracting_medicines(p_data, col_index):
            v_data = p_data
            try:
                # Creating an empty list for saving the parsed data
                v_medicines = []
                # Parsing the data and extracting medicines
                for idx, entry in enumerate(v_data.split(';')):
                    try:
                        # Splitting entry by colon to separate name and dosage
                        parts = entry.split(':')
                        if len(parts) > 1:
                            # Extracting medicine from the first part before colon
                            medicine = parts[0].strip()
                            v_medicines.append(medicine)
                        else:
                            # If no colon, consider medicine as null
                            v_medicines.append('null')
                    except IndexError:
                        print(f"Error processing entry at index {idx} in column {col_index}: {entry}. Skipping.")
                # Joining the extracted medicines into a single string
                return ', '.join(v_medicines)
            except Exception as e:
                print(e)
        
        # Create a function for creating a clinical notes column
        def f_clinical_notes(p_df):
            try:
                df1 = p_df
                v_df = df1

                # Getting the Visit, Gender and age columns
                v_vga_df = v_df[['Visit_id', 'Gender', 'Age']].copy()

                # Replacing the gender the values
                v_vga_df['Gender'] = v_vga_df['Gender'].map({'M': 'Male', 'F': 'Female'})

                # Adding 'Years' to the 'age' column
                v_vga_df['Age'] = v_vga_df['Age'].astype(str) + ' years'

                v_cc_df = v_df[['Visit_id', 'Gender', 'Age', 'Sbp', 'Dbp', 'Pulse', 'Temperature', 'Weight',
                   'Height', 'RR', 'SPO2', 'HB', 'Sugar_random', 'Blood_group', 'Sugar_pp',
                   'Sugar_after_meal', 'Chief_complaint', 'Physical_examination',
                   'Patient_medical_history', 'Family_history']].copy()

                v_cc_df1 = v_cc_df.copy()

                v_cc_df1 = v_cc_df1[['Visit_id', 'Chief_complaint', 'Physical_examination',
                   'Patient_medical_history', 'Family_history']]

                v_df1 = pd.merge(v_vga_df, v_cc_df1, on='Visit_id') #, how='outer')

                v_vitals_df4 = v_cc_df[['Visit_id', 'Sbp', 'Dbp', 'Pulse', 'Temperature', 'Weight',
                   'Height', 'RR', 'SPO2', 'HB', 'Sugar_random', 'Blood_group', 'Sugar_pp',
                   'Sugar_after_meal']].copy()

                # Adding 'Temperature' to the 'Height' column
                v_vitals_df4['Temperature'] = v_vitals_df4['Temperature'].astype(str) + " 'C"

                # Adding 'cm' to the 'Height' column
                v_vitals_df4['Height'] = v_vitals_df4['Height'].astype(str) + ' cm'

                # Adding 'Kg' to the 'weight' column
                v_vitals_df4['Weight'] = v_vitals_df4['Weight'].astype(str) + ' Kg'

                v_df2 = pd.merge(v_df1, v_vitals_df4, on='Visit_id') #, how='outer')

                v_main_cols = v_df2.columns
                #print(len(v_main_cols), type(v_main_cols), v_main_cols)

                for i in v_main_cols:
                    #print(i)
                    v_df2[i] = v_df2[i].fillna('Null')

                v_main_cols = v_df2.columns
                #print(len(v_main_cols), type(v_main_cols), v_main_cols)

                for i in v_main_cols:
                    #print(i)
                    v_df2[i] = v_df2[i].astype(str)

                v_main_cols = v_df2.columns
                #print(len(v_main_cols), type(v_main_cols), v_main_cols)

                for i in v_main_cols:
                    #print(i)
                    if i == 'Visit_id':
                        pass
                    else:
                        v_df2[i] = v_df2[i].apply(h2h.html2text)

                v_cc_df2 = v_df2.copy()

                # Using apply method to combine values along with column names
                v_cc_df2['Clinical_notes'] = v_cc_df2[v_cc_df2.columns[1:]].apply( lambda x: ' '.join([f'{col}: {val}' for col, val in zip(x.index, x.dropna().astype(str))]), axis=1)            

                # Define the vitals information
                vitals_info = "Vitals:- "

                # Function to add vitals before SBP line
                def add_vitals(row):
                    # Find the position of "SBP" in the text
                    sbp_position = row['Clinical_notes'].find("Sbp:")

                    # Insert vitals information before the "SBP" line
                    modified_text = row['Clinical_notes'][:sbp_position] + vitals_info + "\n\n" + row['Clinical_notes'][sbp_position:]

                    return modified_text

                # Apply the add_vitals function to each row of the DataFrame
                v_cc_df2['Clinical_notes'] = v_cc_df2.apply(add_vitals, axis=1)

                v_df5 = v_cc_df2.copy()

                # Remove the line from the specified column
                v_df5['Clinical_notes'] = v_df5['Clinical_notes'].str.replace("•Do you have a family history of any of the following?\*", "") #, regex=True)

                v_df5['Visit_id'] = v_df5['Visit_id'].astype('int64')

                v_df6 = v_df5[['Visit_id','Clinical_notes']].copy()

                v_df6 = v_df6.drop_duplicates()
                #df2 = df1.copy()

                df2 = pd.merge(df1, v_df6, on=['Visit_id'], how='right')
                #df2 = pd.concat([df1, v_df5], axis=1)

                df2.insert(36, 'Clinical_notes1', df2['Clinical_notes'])
                df2 = df2.drop(columns='Clinical_notes', axis=1)
                df2.rename(columns={'Clinical_notes1':'Clinical_notes'}, inplace=True)

                print('Clinical notes added in the final dataframe\n')
                
                # Get the current date and time
                file_name = datetime.now()
                # Format the datetime object
                formatted_file_name = int(file_name.strftime("%Y%m%d%H%M%S"))
                
                # It'll create the filename with date and time
                df2.to_excel(f'PWT_with_Clinical_notes_{formatted_file_name}.xlsx', index=False)
                
                print(f'Final "PWT_with_Clinical_notes_{formatted_file_name}.xlsx" excel output created...:)\n')
                
                #print('Final excel output created\n')
                return df2
            
            except Exception as e:
                print('Error in clinical notes function: ', e)

        ## Function for segregation, we can simply pass the dataframe and get the output
        def f_data_segregation(v_sample):
            try:
                merged_df = v_sample
                
                # Getting the required columns in order for the new one
                v_df_new = merged_df[['Visit_id','Patient_id','OpenMRS_id','Gender','Date_of_birth','Age','Patient_name','Location_state','Location_district','Location_sanch','Location_village','Visit_started_date','Visit_creation_date','Visit_ended_date','CHW_name','Specialty','Visit_upload_time','Chief_complaint','Consultation_start_time','Doctor_name','Doctor_TAT','Patient_TAT','Diagnosis','Medications','Medical_test','Medical_advice','Notes','Follow_up_date','Patient_medical_history','Family_history','Sign_submit_time', 'Patient_waiting_time_(in_hours)']]
                
                ## Here getting the remaining columns
                v_df_additional = merged_df.drop(['Patient_id','OpenMRS_id','Gender','Date_of_birth','Age','Patient_name','Location_state','Location_district','Location_sanch','Location_village','Visit_started_date','Visit_creation_date','Visit_ended_date','CHW_name','Specialty','Visit_upload_time','Chief_complaint','Consultation_start_time','Doctor_name','Doctor_TAT','Patient_TAT','Diagnosis','Medications','Medical_test','Medical_advice','Notes','Follow_up_date','Patient_medical_history','Family_history','Sign_submit_time', 'Patient_waiting_time_(in_hours)'], axis=1)
                
                ## Here merged the two dataframe
                v_df_merged = pd.merge(v_df_new, v_df_additional, on='Visit_id')
                
                # copy of the dataframe
                v_df_merged1 = v_df_merged.copy()

                # copy the datafrmae
                df_cc=v_df_merged1.copy()

                # selecting the required columns for 'chief complaint' to make it as a dataframe
                df_cc = df_cc[['Visit_id','Patient_id','Visit_started_date','Follow_up_date', 'Chief_complaint']]

                #Imputing the nan values in the chief complaint data with null
                df_cc['Chief_complaint'] = df_cc['Chief_complaint'].fillna('null')

                #Removing the unwanted symbols in the chief complaint data
                df_cc['Chief_complaint'] = df_cc['Chief_complaint'].replace('\xa0', ' ', regex=True)


                #Applying the function to the dataframe
                df_cc['Symptoms'] = df_cc['Chief_complaint'].apply(f_combined_extraction)

                #Applying the function to the dataframe
                df_cc['Associated_Symptoms'] = df_cc['Chief_complaint'].apply(f_extract_associated_symptoms)


                # copy the dataframe
                df_diag=v_df_merged1.copy()

                # selecting the required columns for 'Diagnosis' to make it as a dataframe
                df_diag=df_diag[['Visit_id','Patient_id','Visit_started_date','Diagnosis']]


                # Split the strings in the 'Diagnosis' column at the ':' and ',' symbol and extract the unique strings
                v_diagnosis_unique = df_diag['Diagnosis'].str.split(':',expand=True)[1].str.split(',',expand=True)[0].str.strip().dropna().unique()
                # print(v_diagnosis_unique)

                # Create a new columns for each unique diagnosis category
                for diag in v_diagnosis_unique:
                    df_diag[diag] = 'NaN'



                # Columns to match in the 'Diagnosis' column values
                v_columns_to_match = ['Primary & Provisional', 'Secondary & Confirmed', 'Primary & Confirmed', 'Secondary & Provisional']
                #Iterate through each column
                for col in v_columns_to_match:
                    # Extract values before the first ':' using regular expression and also in between ':' as well.
                    df_diag[col] = df_diag['Diagnosis'].apply(lambda x: ''.join(re.findall(fr'([^:]+):{col}', str(x))) if re.search(fr'([^:]+):{col}', str(x)) else None)

                # Remove the specified values from the specified columns
                v_values_to_remove = ['Primary & Provisional,', 'Secondary & Confirmed,', 'Primary & Confirmed,', 'Secondary & Provisional,']


                for col in v_columns_to_match:
                    df_diag[col] = df_diag[col].str.replace('|'.join(v_values_to_remove), '', regex=True)


                # changing the columns order
                df_diag = df_diag.reindex(columns=['Visit_id', 'Patient_id', 'Visit_started_date', 'Diagnosis',
                       'Primary & Provisional', 'Secondary & Provisional','Primary & Confirmed','Secondary & Confirmed'])


                # copy the dataframe
                df_medi=v_df_merged1.copy()

                # Selecting the necessary columns
                df_medi = df_medi[['Visit_id','Patient_id','Visit_started_date','Medications']]

                df_medi['Medications'].isna().sum()

                # Filling the nan values with Null in 'Medication' column
                df_medi['Medications'] = df_medi['Medications'].fillna('Null')

                df_medi['Medications'].isna().sum()

                # Applying the function to the 'Medications' column of DataFrame
                df_medi['Medicines'] = df_medi.apply(lambda row: f_extracting_medicines(row['Medications'], row.name), axis=1)

                # Applying the function to the DataFrame
                df_medi['Strength'] = df_medi.apply(lambda row: f_extracting_strength(row['Medications'], row['Visit_id'], row.name), axis=1)

                # Applying the function to the DataFrame
                df_medi['Dosage'] = df_medi.apply(lambda row: f_extracting_dosage(row['Medications'], row['Visit_id'], row.name), axis=1)


                #remove duplicates
                df_cc = df_cc.drop_duplicates(['Visit_id','Patient_id', 'Visit_started_date'])
                df_diag = df_diag.drop_duplicates(['Visit_id','Patient_id', 'Visit_started_date'])
                df_medi=df_medi.drop_duplicates(['Visit_id','Patient_id', 'Visit_started_date'])


                # Merge "chief complaint" and "Diagnosis" DataFrames based on 'visit_id', 'patient_id', 'Visit_Started_Date' 
                segg_merged_df = pd.merge(df_cc, df_diag, on=['Visit_id', 'Patient_id','Visit_started_date'], how='inner')

                # Merge " merged_df" and "Medications" DataFrames based on 'visit_id', 'patient_id', 'Visit_Started_Date'
                final_segg_df=pd.merge(segg_merged_df, df_medi, on=['Visit_id', 'Patient_id', 'Visit_started_date'], how='inner')

                # removing some columns before merging with the "final_sql_df" to avoid duplicate rows
                final_segg_df.drop(columns=['Follow_up_date','Chief_complaint','Diagnosis','Medications'],axis=1,inplace=True)

                # Merge final_sql_df and final_df using 'inner' join
                complete_df = pd.merge(v_df_merged1, final_segg_df, on=['Visit_id', 'Patient_id', 'Visit_started_date'], how='inner')
                
                # Drop the duplicates
                complete_df = complete_df.drop_duplicates()
                
                # Rearrange the columns
                v_last_df = complete_df[['Visit_id', 'Patient_id','OpenMRS_id','Gender','Date_of_birth','Age','Patient_name','Location_state','Location_district','Location_sanch','Location_village',
                                         'Visit_started_date','Visit_creation_date','Visit_ended_date','CHW_name','Specialty','Visit_upload_time','Sbp','Dbp','Pulse','Temperature','Weight','Height','RR','SPO2','HB','Sugar_random','Blood_group','Sugar_pp','Sugar_after_meal',
                                         'Symptoms','Associated_Symptoms','Chief_complaint','Physical_examination','Patient_medical_history','Family_history','Consultation_start_time','Doctor_name','Doctor_TAT',
                                        'Patient_TAT','Diagnosis_provided','Diagnosis','Primary & Provisional','Secondary & Provisional','Primary & Confirmed','Secondary & Confirmed','Medications','Medicines','Strength','Dosage',
                                        'Medical_test','Medical_advice','Notes','Follow_up_date','Sign_submit_time','Patient_waiting_time_(in_hours)','Creator_Id','Doctor_id','Additional_comments','Feedback','Comments']].copy()
                
                # Export the dataframe to excel
                #v_last_df.to_excel('Final_df1.xlsx',index=False)
                print('Segregated columns added\n')
                # It returns the final dataframe
                return v_last_df

            except Exception as e:
                print('Error in the data segregation function', e)


        ## SQL query to fetch the minimum and maximum dates from "visit" table
        v_min_date = v_min_table_date
        v_last_date = v_max_table_date
        v_hours = v_min_date.hour
        v_minutes = v_min_date.minute
        v_seconds = v_min_date.second

        print(f'Date Between **Starting Date - ({v_min_date}) and **End Date - ({v_last_date}).\n\n**The format is yyy-mm-dd\n')

        try:
            while True:
                # Getting input from user
                v_strt_date_input = v_visit_started_date

                pd.to_datetime(v_strt_date_input, format='%Y-%m-%d')
                #v_start_date = v_strt_date_input+' '+str(v_hours)+':'+str(v_minutes)+':'+str(v_seconds)
                v_start_date = v_strt_date_input+' 00:00:00'
                date_format = '%Y-%m-%d %H:%M:%S'

                # Convert the string to a datetime object
                datetime_object = datetime.strptime(v_start_date, date_format)
                print("\nDate Entered: \t",datetime_object)

                try:
                    if (datetime_object<=v_last_date): # (datetime_object>=v_min_date) and 
                        while True:
                            # Getting input from user
                            v_end_date_input = v_visit_ended_date
                            pd.to_datetime(v_end_date_input, format='%Y-%m-%d')
                            v_end_date = v_end_date_input+' 23:59:59'

                            #v_end_date = v_end_date+' 00:00:00'
                            # Convert the string to a datetime object
                            datetime_object1 = datetime.strptime(v_end_date, date_format)
                            print("\nDate Entered: \t",datetime_object1,'\n')

                            try:
                                if (datetime_object1<v_last_date) or (datetime_object1>datetime_object):
                                    if datetime_object1 < datetime_object:
                                        print('Incorrct Date. Please Enter the Date within the range.')
                                    else:
                                        # Getting the username
                                        v_hostname = socket.gethostname()
                                        v_host_name.append(v_hostname)

                                        # Entered data ranges
                                        date_range.append(v_start_date+' / '+v_end_date)
                                        
                                        # Passing the startdate, end date and query in the below function to fetch the data
                                        #v_sample = f_filter_data(datetime_object, datetime_object1, v_queries)
                                        
                                        # Execute each query with input parameters
                                        v_params = (v_start_date, v_end_date)
                                        
                                        #connect database and cursor     
                                        v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)
                                            
                                        # Execute each query with input parameters and store the output in a dictionary
                                        v_output_dataframes = {}
                                        for query_name, query in v_queries.items():
                                            
                                            df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                                            
                                            if df is not None:
                                                v_output_dataframes[query_name] = df
                                                print(f"{query_name} executed successfully.\n")

                                        # Close the database connection
                                        f_close_database_connection(v_mydb, v_cursor)

                                        # Merge all the data frames based on 'visit_id' using 'outer' join
                                        merged_df = None
                                        for df in v_output_dataframes.values():
                                            if merged_df is None:
                                                merged_df = df
                                            else:
                                                merged_df = pd.merge(merged_df, df, on='Visit_id', how='outer')
                                        
                                        v_sample = merged_df.copy()
                                        print('\nCombined dataframe created\n')
                                        
                                        # After fetching the data need to calculate the new column Patient_waiting_time_(in_hours)
                                        
                                        # Convert the 'Visit_started_date' and 'Sign_submit_time' columns to datetime objects
                                        v_sample['Visit_started_date'] = pd.to_datetime(v_sample['Visit_started_date'])
                                        v_sample['Sign_submit_time'] = pd.to_datetime(v_sample['Sign_submit_time'])


                                        # Calculate the time difference in hours
                                        print('Patient_waiting_time_(in_hours) column added\n')
                                        v_sample['Patient_waiting_time_(in_hours)'] = (v_sample['Sign_submit_time'] - v_sample['Visit_started_date']).dt.total_seconds() / 3600
                                        #v_sample.to_excel('Demo2.xlsx', index=False)
                                        
                                        # After getting Patient_waiting_time_(in_hours) this column pass the dataframe to segregation
                                        df_segregate = f_data_segregation(v_sample)
                                        
                                        # After segregated columns we've to make a clinical notes column
                                        final_output = f_clinical_notes(df_segregate)

                                        end = datetime.now()
                                        ed.append(end)

                                        duration = ed[-1]-st[-1]
                                        du.append(duration)

                                        dur = {'Host_name':v_host_name, 'Date_Range':date_range, 'Start': st, 'End':ed, 'Duration':du}
                                        duration_df = pd.DataFrame(dur)

                                        # Convert the 'Duration' column to seconds
                                        duration_df['Duration'] = duration_df['Duration'].dt.total_seconds()
                                        duration_df.to_excel('Duration.xlsx', index=False)
                                        
                                        #print('Executed successfully ')
                                        
                                        # It returns the segregated dataframe and the duration dataframe
                                        
                                        return final_output, duration_df
                                        
                                else:
                                    print('Enter the date with range')
                            except Exception as e:
                                print('Error 2', e)
                                break
                        break
                    else:
                        print('Error3')
                    #break
                except Exception as e:
                            print(e)
        except Exception as e:
            print('Error 4', e)
    except Exception as e:
        print('Error in the main function: ', e)

In [3]:
v_data_seg = f_final_function('2023-05-12', '2024-03-20')

Enter the host: afitraining.ekalarogya.org
Enter the username: root
Enter the password: ········
Enter the database: openmrs
Connected to the database

Cursor closed.
Database connection closed.

Connected to the database

Cursor closed.
Database connection closed.

Date Between **Starting Date - (2023-05-12 13:48:58) and **End Date - (2024-03-18 11:24:49).

**The format is yyy-mm-dd


Date Entered: 	 2023-05-12 00:00:00

Date Entered: 	 2024-03-20 23:59:59 

Connected to the database

Query1 executed successfully.

Query2 executed successfully.

Query3 executed successfully.

Query4 executed successfully.

Query5 executed successfully.

Query6 executed successfully.

Query7 executed successfully.

Cursor closed.
Database connection closed.


Combined dataframe created

Patient_waiting_time_(in_hours) column added

Segregated columns added

Clinical notes added in the final dataframe

Final "PWT_with_Clinical_notes_20240321172426.xlsx" excel output created...:)



In [4]:
v_df = v_data_seg[0]
v_df

,Visit_id,Patient_id,OpenMRS_id,Gender,Date_of_birth,Age,Patient_name,Location_state,Location_district,Location_sanch,Location_village,Visit_started_date,Visit_creation_date,Visit_ended_date,CHW_name,Specialty,Visit_upload_time,Sbp,Dbp,Pulse,Temperature,Weight,Height,RR,SPO2,HB,Sugar_random,Blood_group,Sugar_pp,Sugar_after_meal,Symptoms,Associated_Symptoms,Chief_complaint,Physical_examination,Patient_medical_history,Family_history,Clinical_notes,Consultation_start_time,Doctor_name,Doctor_TAT,Patient_TAT,Diagnosis_provided,Diagnosis,Primary & Provisional,Secondary & Provisional,Primary & Confirmed,Secondary & Confirmed,Medications,Medicines,Strength,Dosage,Medical_test,Medical_advice,Notes,Follow_up_date,Sign_submit_time,Patient_waiting_time_(in_hours),Creator_Id,Doctor_id,Additional_comments,Feedback,Comments
0,1,481044,2AEVC-4,M,1992-05-12,32,Test Bhushan Offline Mode 1,Maharashtra,Nashik,Jogmudi,Anbe,2023-05-12 13:48:58,2023-05-12 13:54:58,NaT,nurse1,General Physician,12 May 2023 13:54:53,55.0,56.0,NaN,NaN,65.0,175.0,NaN,NaN,None,None,None,None,None,Cough,Null,►<b>Cough</b>: <br/>• Duration - Subacute ( 2 ...,<b>General exams: </b><br/>• Eyes: Jaundice-no...,• Medical History - Tuberculosis - Treatment -...,•Do you have a family history of any of the fo...,Gender: Male\n\n Age: 32 years\n\n Chief_compl...,None,None,None,5.1000,NaN,NaN,None,None,None,None,NaN,null,null,null,NaN,NaN,NaN,NaN,NaT,NaN,318,NaN,None,NaN,NaN
1,2,481050,2AEVJ-8,M,2003-05-12,21,Sangana Test,Maharashtra,Nashik,Jogmudi,Anbe,2023-05-12 14:44:23,2023-05-12 14:51:08,NaT,nurse1,General Physician,12 May 2023 14:51:02,190.0,80.0,NaN,NaN,80.0,180.0,NaN,NaN,None,None,None,None,None,"Cold, Sneezing",Cough,"►<b>Cold, Sneezing</b>: <br/>• 2 Days.<br/>• P...",<b>General exams: </b><br/>• Eyes: Jaundice-no...,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,Gender: Male\n\n Age: 21 years\n\n Chief_compl...,None,None,None,6.6500,NaN,NaN,None,None,None,None,NaN,null,null,null,NaN,NaN,NaN,NaN,NaT,NaN,318,NaN,None,NaN,NaN
2,3,481053,2AEVM-2,M,1999-05-12,25,shankar Test 456,Jharkhand,Giridih,Parasnath,Chatro,2023-05-12 16:16:11,2023-05-12 16:22:45,2023-05-12 16:28:43,nurse1,General Physician,12 May 2023 16:22:41,NaN,NaN,NaN,NaN,60.0,130.0,NaN,NaN,None,None,None,None,None,Cough,Hemoptysis,►<b>Cough</b>: <br/>• Duration - Subacute ( 2 ...,<b>General exams: </b><br/>• Eyes: Jaundice-<b...,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,Gender: Male\n\n Age: 25 years\n\n Chief_compl...,2023-05-12 16:25:14,Super User,4.4000,12.5333,yes,Cough:Primary & Provisional,Cough,None,None,None,None,null,null,null,None,None,None,None,2023-05-12 16:27:09,0.182778,318,1.0,None,None,None
3,4,481053,2AEVM-2,M,1999-05-12,25,shankar Test 456,Jharkhand,Giridih,Parasnath,Chatro,2023-05-12 16:23:53,2023-05-12 16:30:22,NaT,nurse1,General Physician,12 May 2023 16:30:17,NaN,NaN,NaN,NaN,60.0,130.0,NaN,NaN,None,None,None,None,None,Dry mouth,Loss of saliva and tears,►<b>Dry mouth</b>: <br/>• Duration - 2 Days.<b...,<b>General exams: </b><br/>• Eyes: Jaundice-<b...,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,Gender: Male\n\n Age: 25 years\n\n Chief_compl...,None,None,None,5.3500,NaN,NaN,None,None,None,None,NaN,null,null,null,NaN,NaN,NaN,NaN,NaT,NaN,318,NaN,None,NaN,NaN
4,5,481160,2AF07-6,M,1997-05-15,27,Test Test,Madhya Pradesh,Khargone,Karhi,Bawi Tada,2023-05-15 13:49:02,2023-05-15 13:55:22,NaT,nurse10,General Physician,15 May 2023 13:55:21,NaN,NaN,NaN,NaN,65.0,180.0,NaN,NaN,None,None,None,None,None,Diarrhea,Null,►<b>Diarrhea</b>: <br/>• Duration - 94 Hours.<...,<b>General exams: </b><br/>• Eyes: Jaundice-no...,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,Gender: Male\n\n Age: 27 years\n\n Chief_compl...,None,None,None,6.3167,NaN,NaN,None,None,None,None,NaN,null,null,null,NaN,NaN,NaN,NaN,NaT

In [5]:
duration_df = v_data_seg[1]

In [6]:
duration_df

,Host_name,Date_Range,Start,End,Duration
0,JOHAN-PC,2023-05-12 00:00:00 / 2024-03-20 23:59:59,2024-03-21 17:20:37.594990,2024-03-21 17:24:29.206040,231.61105
